In [ ]:
# Step1: Generate Head Dataset and Velocity Dataset
import numpy as np
import os
import pandas as pd
import h5py
from numba import jit, prange
import plotly.graph_objects as go

# Set parameters for the simulation
params = {
    'HRx': 6, 'HRy': 3, 'HLx': 3, 'HLy': 0,
    'LX': 10000, 'LY': 10000, 'D': -2000, 'p': 3, 'q': 3
}

# Extract parameters from the dictionary
HRx, HRy, HLx, HLy = params['HRx'], params['HRy'], params['HLx'], params['HLy']
LX, LY, D, p, q = params['LX'], params['LY'], params['D'], params['p'], params['q']

# Define the grid resolution
mesh_x, mesh_y, mesh_z = 101, 101, 101
x_range = np.linspace(0, LX, mesh_x)
y_range = np.linspace(0, LY, mesh_y)
z_range = np.linspace(D, 0, mesh_z)

# Pre-compute constants to optimize performance
cosh_D_LX = np.cosh(np.pi * D / LX)
cosh_D_LY = np.cosh(np.pi * D / LY)
pi_D_LX = np.pi * D / LX
pi_D_LY = np.pi * D / LY

# Function to calculate the velocity components (u, v, w) and hydraulic head (h)
@jit(nopython=True)
def compute_velocity(x, y, z, LX, LY, D, HRx, HRy, HLx, HLy, p, q):
    # Calculate velocity components u, v, w
    u = -(np.pi / LX * HRx * np.sin(np.pi * x / LX) * np.cosh(np.pi * (z - D) / LX) / np.cosh(np.pi * D / LX) +
         p * np.pi / LX * HLx * np.sin(p * np.pi * x / LX) * np.cosh(p * np.pi * (z - D) / LX) / np.cosh(p * np.pi * D / LX))
    
    v = -(np.pi / LY * HRy * np.sin(np.pi * y / LY) * np.cosh(np.pi * (z - D) / LY) / np.cosh(np.pi * D / LY) +
         q * np.pi / LY * HLy * np.sin(q * np.pi * y / LY) * np.cosh(q * np.pi * (z - D) / LY) / np.cosh(q * np.pi * D / LY))
    
    w = (np.pi / LX * HRx * np.cos(np.pi * x / LX) * np.sinh(np.pi * (z - D) / LX) / np.cosh(np.pi * D / LX) +
         p * np.pi / LX * HLx * np.cos(p * np.pi * x / LX) * np.sinh(p * np.pi * (z - D) / LX) / np.cosh(p * np.pi * D / LX) +
         np.pi / LY * HRy * np.cos(np.pi * y / LY) * np.sinh(np.pi * (z - D) / LY) / np.cosh(np.pi * D / LY) +
         q * np.pi / LY * HLy * np.cos(q * np.pi * y / LY) * np.sinh(q * np.pi * (z - D) / LY) / np.cosh(q * np.pi * D / LY))
    
    # Calculate the hydraulic head h
    h = HRx - HRx * np.cos(np.pi * x / LX) * np.cosh(np.pi * (z - D) / LX) / np.cosh(np.pi * D / LX) + \
        HRy - HRy * np.cos(np.pi * y / LY) * np.cosh(np.pi * (z - D) / LY) / np.cosh(np.pi * D / LY) + \
        HLx - HLx * np.cos(p * np.pi * x / LX) * np.cosh(p * np.pi * (z - D) / LX) / np.cosh(p * np.pi * D / LX) + \
        HLy - HLy * np.cos(q * np.pi * y / LY) * np.cosh(q * np.pi * (z - D) / LY) / np.cosh(q * np.pi * D / LY)
    
    return u, v, w, h

# Initialize velocity fields and water table elevation arrays
vx = np.zeros((mesh_x, mesh_y, mesh_z))
vy = np.zeros((mesh_x, mesh_y, mesh_z))
vz = np.zeros((mesh_x, mesh_y, mesh_z))
h = np.zeros((mesh_x, mesh_y, mesh_z))
zwt = np.zeros((mesh_x, mesh_y))

# Function to calculate the velocity field over the entire grid
@jit(nopython=True, parallel=True)
def calculate_velocity_field(vx, vy, vz, h, zwt, x_range, y_range, z_range, LX, LY, D, HRx, HRy, HLx, HLy, p, q):
    for i in prange(mesh_x):
        for j in range(mesh_y):
            for k in range(mesh_z):
                vx[i, j, k], vy[i, j, k], vz[i, j, k], h[i, j, k] = compute_velocity(x_range[i], y_range[j], z_range[k], LX, LY, D, HRx, HRy, HLx, HLy, p, q)
            # Calculate the water table elevation (head at z = 0)
            zwt[i, j] = h[i, j, -1]

# Compute the velocity field
calculate_velocity_field(vx, vy, vz, h, zwt, x_range, y_range, z_range, LX, LY, D, HRx, HRy, HLx, HLy, p, q)

# Calculate the magnitude of the velocity field
UU = np.sqrt(vx**2 + vy**2 + vz**2)

# Function to save data to an HDF5 file
def save_data_hdf5(filename, vx, vy, vz, h, UU, zwt, x_range, y_range, z_range):
    with h5py.File(filename, 'w') as f:
        f.create_dataset('vx', data=vx)
        f.create_dataset('vy', data=vy)
        f.create_dataset('vz', data=vz)
        f.create_dataset('h', data=h)
        f.create_dataset('UU', data=UU)
        f.create_dataset('zwt', data=zwt)  # Save zwt
        f.create_dataset('x_range', data=x_range)
        f.create_dataset('y_range', data=y_range)
        f.create_dataset('z_range', data=z_range)

# Function to save data to CSV files
def save_data_csv(filename, vx, vy, vz, h, UU, zwt, x_range, y_range, z_range):
    data = []
    for i in range(len(x_range)):
        for j in range(len(y_range)):
            for k in range(len(z_range)):
                data.append([x_range[i], y_range[j], z_range[k], vx[i, j, k], vy[i, j, k], vz[i, j, k], h[i, j, k], UU[i, j, k]])
    df = pd.DataFrame(data, columns=['x', 'y', 'z', 'vx', 'vy', 'vz', 'h', 'UU'])
    zwt_df = pd.DataFrame(zwt, index=x_range, columns=y_range)
    df.to_csv(filename, index=False)
    zwt_df.to_csv(filename.replace('.csv', '_zwt.csv'))  # Save zwt as a separate CSV file

# Determine the current working directory
current_working_dir = os.getcwd()

# Define the results folder path
result_folder = os.path.join(current_working_dir, 'results')
os.makedirs(result_folder, exist_ok=True)

# Define output paths for HDF5 and CSV files
output_path_hdf5 = os.path.join(result_folder, 'head_and_velocity.h5')
output_path_csv = os.path.join(result_folder, 'head_and_velocity.csv')

# Save data to files
save_data_hdf5(output_path_hdf5, vx, vy, vz, h, UU, zwt, x_range, y_range, z_range)
save_data_csv(output_path_csv, vx, vy, vz, h, UU, zwt, x_range, y_range, z_range)

# Create a DataFrame for statistical analysis
df = pd.DataFrame({
    'vx': vx.flatten(),
    'vy': vy.flatten(),
    'vz': vz.flatten(),
    'h': h.flatten(),
    'UU': UU.flatten()
})

# Set display format to scientific notation with two decimal places
pd.options.display.float_format = '{:.2e}'.format

# Perform and print statistical description of the data
description = df.describe()
print(description)

# Plot the 3D distribution of the water table elevation (zwt)
fig = go.Figure(data=[go.Surface(
    z=zwt,
    x=x_range,
    y=y_range,
    colorscale='Rainbow',
    colorbar=dict(title='zwt', tickfont=dict(size=12))   
)])
fig.update_layout(
    title='3D Water Table Elevation Distribution',
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Water Table Elevation'
    )
)

# Save the plot as an HTML file and display it
output_html_path = os.path.join(result_folder, 'Water_table_Case3S.html')
fig.write_html(output_html_path)
fig.show()

def save_data_tecplot(filename, vx, vy, vz, UU, h, x_range, y_range, z_range):
    """
    Saves the velocity components and hydraulic head in a Tecplot-readable DAT file.
    """
    # Create a mesh grid of the coordinates
    x, y, z = np.meshgrid(x_range, y_range, z_range, indexing='ij')
 
 
    # Flatten the arrays
    x_flat = x.flatten()
    y_flat = y.flatten()
    z_flat = z.flatten()
    vx_flat = vx.flatten()
    vy_flat = vy.flatten()
    vz_flat = vz.flatten()
    UU_flat = UU.flatten()
    h_flat = h.flatten()
 
    # Create a mask for z values greater than or equal to 0
    mask = z_flat >= 1
    # Stack data column-wise
    data = np.column_stack((x_flat, y_flat, z_flat, vx_flat, vy_flat, vz_flat, UU_flat, h_flat))

    # Sort data by x, then y, then z
    data = data[np.lexsort((z_flat, y_flat, x_flat))]

    # Write the header information and data to the file
    with open(filename, 'w') as f:
        f.write('VARIABLES = "X", "Y", "Z", "VX", "VY", "VZ", "UU", "h"\n')
        f.write('ZONE T="3D Flow Field", I={}, J={}, K={}, F=POINT\n'.format(mesh_x, mesh_y, mesh_z))
        np.savetxt(f, data, fmt='%1.6e', delimiter=', ')

# Ensure the call to save_data_tecplot uses the correct parameters
output_path_tecplot = os.path.join(result_folder, 'head_and_velocity.dat')
save_data_tecplot(output_path_tecplot, vx, vy, vz, UU, h, x_range, y_range, z_range)


# Print completion message with the output file paths
print(f'Data successfully saved to {output_path_hdf5} and {output_path_csv}')
print(f'Tecplot DAT file has been saved to {output_path_tecplot}')
print(f'Water table elevation plot saved to {output_html_path}')


In [ ]:
# Step2: Finding Out (Pseudo-)stagnation Points/Lines
import os
from pathlib import Path
import numpy as np
import plotly.graph_objects as go
import h5py
from scipy.interpolate import griddata
from scipy.ndimage import minimum_filter
from concurrent.futures import ThreadPoolExecutor
import pandas as pd

# Get the current working directory
script_dir = Path.cwd()

# Construct the data file path relative to the script directory
file_relative_path = Path('results', 'head_and_velocity.h5')
file_path = script_dir / file_relative_path

# Check if the file exists
if not file_path.exists():
    print(f"Error: File '{file_path}' does not exist!")
    exit()

# Attempt to load the data file
try:
    with h5py.File(file_path, 'r') as f:
        x_range = f['x_range'][:]
        y_range = f['y_range'][:]
        z_range = f['z_range'][:]
        vx = f['vx'][:]
        vy = f['vy'][:]
        vz = f['vz'][:]
        UU = f['UU'][:]
    print("File loaded successfully")
except Exception as e:
    print(f"Error loading file: {e}")
    exit()

# Define the boundaries for filtering
x_min, x_max = 0, 10000
y_min, y_max = 0, 10000
z_min, z_max = -2000, 0

# Create a grid and filter data within the defined range
X, Y, Z = np.meshgrid(x_range, y_range, z_range, indexing='ij')
filtered_indices = (
    (X >= x_min) & (X <= x_max) &
    (Y >= y_min) & (Y <= y_max) &
    (Z >= z_min) & (Z <= z_max)
)
X_filtered = X[filtered_indices]
Y_filtered = Y[filtered_indices]
Z_filtered = Z[filtered_indices]
vx_filtered = vx[filtered_indices]
vy_filtered = vy[filtered_indices]
vz_filtered = vz[filtered_indices]
UU_filtered = UU[filtered_indices]

# Define a function to filter points that are at least 200 meters apart
def filter_points(points):
    filtered = []
    for point in points:
        if all(np.linalg.norm(np.array(point) - np.array(p)) >= 200 for p in filtered):
            filtered.append(point)
    return filtered

# Set thresholds and boundary deltas
threshold = 1e-3
x_bd, y_bd, z_bd = 0, 0, 0

# Define a function to process each y-slice
def process_slice_y(y):
    slice_indices = np.where(Y_filtered == y)
    if len(slice_indices[0]) == 0:
        print(f"No data on the slice at y={y}.")
        return []

    points = np.array([X_filtered[slice_indices], Z_filtered[slice_indices]]).T
    vx_values = vx_filtered[slice_indices]
    vy_values = vy_filtered[slice_indices]
    vz_values = vz_filtered[slice_indices]
    UU_values = UU_filtered[slice_indices]

    grid_x, grid_z = np.mgrid[x_min:x_max:200j, z_min:z_max:200j]

    grid_vx = griddata(points, vx_values, (grid_x, grid_z), method='linear')
    grid_vy = griddata(points, vy_values, (grid_x, grid_z), method='linear')
    grid_vz = griddata(points, vz_values, (grid_x, grid_z), method='linear')
    grid_UU = griddata(points, UU_values, (grid_x, grid_z), method='linear')

    UU_local_min_points_slice = []

    # Detect local minima in the UU grid
    UU_local_min = (grid_UU == minimum_filter(grid_UU, size=50))

    for i in range(1, grid_x.shape[0] - 1):
        for j in range(1, grid_x.shape[1] - 1):
            if UU_local_min[i, j] and x_min + x_bd <= grid_x[i, j] <= x_max - x_bd and z_min + z_bd <= grid_z[i, j] <= z_max - z_bd:
                if grid_UU[i, j] < threshold:
                    UU_local_min_points_slice.append((grid_x[i, j], y, grid_z[i, j], grid_UU[i, j]))

    # Filter points that are too close to each other
    UU_local_min_points_slice = filter_points(UU_local_min_points_slice)

    return UU_local_min_points_slice


# Parallel processing of y and x slices
y_slices = np.arange(y_min, y_max + 1, 100)

UU_local_min_points_y = []


# Use ThreadPoolExecutor to speed up processing
with ThreadPoolExecutor() as executor:
    results_y = executor.map(process_slice_y, y_slices)
    for res in results_y:
        UU_local_min_points_y.extend(res)

# Aggregate and filter points from all slices
def compare_and_filter_points(points):
    filtered = []
    for point in points:
        existing_point = next((p for p in filtered if np.linalg.norm(np.array(point[:3]) - np.array(p[:3])) < 1), None)
        if existing_point:
            if point[3] < existing_point[3]:
                filtered.remove(existing_point)
                filtered.append(point)
        else:
            filtered.append(point)
    return filtered

# Sort points by their x-coordinate
def sort_points_by_x(points):
    return sorted(points, key=lambda point: point[0])

# Save filtered and sorted points to CSV files
def save_points_to_csv(points, file_name, header):
    csv_path = script_dir / 'results' / file_name
    with open(csv_path, 'w') as f:
        f.write(header)
        for point in points:
            f.write(', '.join(map(str, point)) + '\n')
    print(f"Saved {len(points)} points to {file_name}")



# Save results to CSV files
save_points_to_csv(UU_local_min_points_y, 'best_y.csv', 'x, y, z, UU\n')

# Generate DAT files from CSV files for visualization
def process_csv_file(file_prefix):
    input_path = os.path.join('results', f'{file_prefix}.csv')
    df = pd.read_csv(input_path)
    
    # Ensure column names are consistent
    df.columns = ['x', 'y', 'z', df.columns[-1]]
    
    output_path_dat = os.path.join('results', f'{file_prefix}.dat')
    df[['x', 'y', 'z']].to_csv(output_path_dat, sep='\t', index=False, header=False)
    with open(output_path_dat, 'r+') as f:
        content = f.read()
        f.seek(0, 0)
        f.write('VARIABLES = "X", "Y", "Z"\n')
        f.write(f"ZONE T='{file_prefix}'\n")
        f.write(f'I={len(df)}, J=1, K=1, F=POINT\n')
        f.write(content)
    
    print(f"Generated {file_prefix}.dat files")
   
   
# Process the generated CSV files
process_csv_file('best_y')

# Visualize the data using Plotly
def visualize_data(csv_file, title):
    df = pd.read_csv(os.path.join('results', csv_file), skipinitialspace=True)

    fig = go.Figure(data=go.Scatter3d(
        x=df['x'],
        y=df['y'],
        z=df['z'],
        mode='markers',
        marker=dict(
            size=5,
            color=df['UU'],   # Color by the UU value
            colorscale='Rainbow', # Use a rainbow color scale
            opacity=0.8
        )
    ))

    fig.update_layout(
        title=title,
        scene=dict(
            xaxis_title='X',
            yaxis_title='Y',
            zaxis_title='Z'
        )
    )
    
    # Update layout to expand display range
    fig.update_layout(
        scene=dict(
            xaxis=dict(range=[x_min, x_max], title='X Axis'),
            yaxis=dict(range=[y_min, y_max], title='Y Axis'),
            zaxis=dict(range=[z_min, z_max], title='Z Axis'),
            aspectmode='manual',
            aspectratio=dict(x=1, y=1, z=0.5)
        ),
        margin=dict(l=0, r=0, b=0, t=0)
    )
    
    output_html_path = os.path.join('results', f'{csv_file.split(".")[0]}.html')
    fig.write_html(output_html_path)
    print(f"Visualization saved as {output_html_path}")

# Generate and visualize plots for each CSV file
visualize_data('best_y.csv', 'Local Minima in Y Slices')


print("All processes completed successfully.")
# Generate visualizations of points
def plot_points(points, title, color):
    fig = go.Figure(data=go.Scatter3d(
        x=[x for x, _, _, _ in points],
        y=[y for _, y, _, _ in points],
        z=[z for _, _, z, _ in points],
        mode='markers',
        marker=dict(size=2, color=color),
    ))

    fig.update_layout(
        title=title,
        scene=dict(
            xaxis_title='X',
            yaxis_title='Y',
            zaxis_title='Z'
        )
    )

    # Update layout to expand display range
    fig.update_layout(
        scene=dict(
            xaxis=dict(range=[x_min, x_max], title='X Axis'),
            yaxis=dict(range=[y_min, y_max], title='Y Axis'),
            zaxis=dict(range=[z_min, z_max], title='Z Axis'),
            aspectmode='manual',
            aspectratio=dict(x=1, y=1, z=0.5)
        ),
        margin=dict(l=0, r=0, b=0, t=0)
    )

    return fig

# Show and save the plots. 
print("Generating visualization for 'best_y.csv'...")
fig_best_y = plot_points(UU_local_min_points_y, 'Best Points from Y Slices with Local Min UU', 'blue')
fig_best_y.show()




In [ ]:
# Step3-1: Generate Critical Points Around (Pseudo-)Stagnation Points along 6 directions
import os
import pandas as pd
import csv
import numpy as np
from scipy.interpolate import griddata

# Function to process a CSV file, convert it, and generate surrounding points
def process_csv_file(file_prefix):
    # Define file paths
    current_working_dir = os.getcwd()
    result_folder = os.path.join(current_working_dir, 'results')
    input_path = os.path.join(result_folder, f'{file_prefix}.csv')

    # Read the CSV file
    df = pd.read_csv(input_path)

    # Check and clean column names
    print(f"Column names ({file_prefix}):", df.columns)
    df.columns = df.columns.str.strip()  # Remove any leading/trailing spaces from column names

    # Ensure the necessary columns are present
    required_columns = {'x', 'y', 'z'}
    if not required_columns.issubset(df.columns):
        raise KeyError(f"Missing required columns: {required_columns - set(df.columns)}")

    # Convert CSV to Tecplot (.dat) format
    output_path_dat = os.path.join(result_folder, f'{file_prefix}.dat')
    with open(output_path_dat, 'w') as f:
        f.write('VARIABLES = "X", "Y", "Z"\n')
        f.write(f"ZONE T='{file_prefix}'\n")
        f.write(f'I={len(df)}, J=1, K=1, F=POINT\n')
        df[['x', 'y', 'z']].to_csv(f, sep=' ', index=False, header=False)
    print(f'File generated: {output_path_dat}')

    # Function to generate surrounding points and save as .csv and .dat files
    def generate_surrounding_points(df, filename_prefix, delta_x_neg, delta_x_pos, delta_y_neg, delta_y_pos, delta_z_neg, delta_z_pos):
        def write_to_dat(df, output_path, zone_title):
            with open(output_path, 'w') as f:
                f.write('VARIABLES = "X", "Y", "Z"\n')
                f.write(f"ZONE T='{zone_title}'\n")
                f.write(f'I={len(df)}, J=1, K=1, F=POINT\n')
                df[['x', 'y', 'z']].to_csv(f, sep=' ', index=False, header=False)

        def create_and_save_copy(df, coordinate, delta, direction):
            df_copy = df.copy()
            df_copy[coordinate] += delta if direction == '+' else -delta
            csv_filename = f'{filename_prefix}_trace_{coordinate}{direction}'
            csv_path = os.path.join(result_folder, f'{csv_filename}.csv')
            dat_path = os.path.join(result_folder, f'{csv_filename}.dat')
            df_copy.to_csv(csv_path, index=False)
            write_to_dat(df_copy, dat_path, csv_filename)
            print(f'Files generated: {csv_path}, {dat_path}')

        # Generate surrounding points for x, y, z directions
        create_and_save_copy(df, 'x', delta_x_neg, '-')
        create_and_save_copy(df, 'x', delta_x_pos, '+')
        create_and_save_copy(df, 'y', delta_y_neg, '-')
        create_and_save_copy(df, 'y', delta_y_pos, '+')
        create_and_save_copy(df, 'z', delta_z_neg, '-')
        create_and_save_copy(df, 'z', delta_z_pos, '+')

    # Generate and save surrounding points
    generate_surrounding_points(df, file_prefix, 300, 300, 300, 300, 200, 200)

    # Define paths for CSV and macro files
    csv_files = [
        os.path.join(result_folder, f'{file_prefix}_trace_x-.csv'),
        os.path.join(result_folder, f'{file_prefix}_trace_x+.csv'),
        os.path.join(result_folder, f'{file_prefix}_trace_y-.csv'),
        os.path.join(result_folder, f'{file_prefix}_trace_y+.csv'),
        os.path.join(result_folder, f'{file_prefix}_trace_z-.csv'),
        os.path.join(result_folder, f'{file_prefix}_trace_z+.csv')
    ]
    macro_files = [
        os.path.join(result_folder, f'{file_prefix}_trace_x-.mcr'),
        os.path.join(result_folder, f'{file_prefix}_trace_x+.mcr'),
        os.path.join(result_folder, f'{file_prefix}_trace_y-.mcr'),
        os.path.join(result_folder, f'{file_prefix}_trace_y+.mcr'),
        os.path.join(result_folder, f'{file_prefix}_trace_z-.mcr'),
        os.path.join(result_folder, f'{file_prefix}_trace_z+.mcr')
    ]

    # Define streamtrace colors
    colors = ['Red', 'Blue', 'Green', 'Yellow', 'Purple', 'Cyan']

    # Function to generate macro files for Tecplot
    def generate_macro_file(csv_file_path, macro_file_path, color):
        points = []
        with open(csv_file_path, mode='r') as csvfile, open(macro_file_path, mode='w') as macrofile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                points.append((float(row['x']), float(row['y']), float(row['z'])))
            macrofile.write('#!MC 1410\n')
            macrofile.write(f'$!StreamAttributes Color = {color}\n')
            for point in points:
                macrofile.write('$!Streamtrace Add\n')
                macrofile.write('  StreamType = VolumeLine\n')
                macrofile.write('  StreamDirection = Both\n')
                macrofile.write('  StartPos\n')
                macrofile.write('    {\n')
                macrofile.write(f'    X = {point[0]}\n')
                macrofile.write(f'    Y = {point[1]}\n')
                macrofile.write(f'    Z = {point[2]}\n')
                macrofile.write('    }\n')
            macrofile.write('$!REDRAWALL\n')
        print(f'Macro file generated: {macro_file_path}')

    # Generate macro files for each direction
    for csv_file, macro_file, color in zip(csv_files, macro_files, colors):
        generate_macro_file(csv_file, macro_file, color)

    print(f'Macro files generated: {", ".join([os.path.basename(file) for file in macro_files])}')

# List of file prefixes to process
file_prefixes = ['best_y']

# Process each file prefix
for prefix in file_prefixes:
    process_csv_file(prefix)


In [ ]:
#Step 3-2: Generate Diving Streamlines Across Critical Points
import numpy as np
import h5py
import os
import pandas as pd
import plotly.graph_objects as go
from scipy.integrate import solve_ivp
from concurrent.futures import ThreadPoolExecutor
from numba import jit

# Function to read data from an HDF5 file
def read_data_hdf5(filename):
    with h5py.File(filename, 'r') as f:
        vx = f['vx'][:]
        vy = f['vy'][:]
        vz = f['vz'][:]
        UU = f['UU'][:]
        x_range = f['x_range'][:]
        y_range = f['y_range'][:]
        z_range = f['z_range'][:]
    return vx, vy, vz, UU, x_range, y_range, z_range

# JIT-compiled velocity field function using Numba for performance optimization
@jit(nopython=True)
def velocity_field_numba(pos, vx, vy, vz, x_range, y_range, z_range):
    x, y, z = pos
    # Check if the point is outside the defined ranges
    if x < x_range[0] or x > x_range[-1] or y < y_range[0] or y > y_range[-1] or z < z_range[0] or z > z_range[-1]:
        return np.array([0.0, 0.0, 0.0])
    
    # Locate the index positions for interpolation
    xi = np.searchsorted(x_range, x) - 1
    yi = np.searchsorted(y_range, y) - 1
    zi = np.searchsorted(z_range, z) - 1

    # Retrieve the boundary points for interpolation
    x1, x2 = x_range[xi], x_range[xi+1]
    y1, y2 = y_range[yi], y_range[yi+1]
    z1, z2 = z_range[zi], z_range[zi+1]

    # Calculate the interpolation factors
    xd = (x - x1) / (x2 - x1)
    yd = (y - y1) / (y2 - y1)
    zd = (z - z1) / (z2 - z1)

    # Trilinear interpolation for velocity components
    vx_val = ((vx[xi, yi, zi] * (1 - xd) + vx[xi + 1, yi, zi] * xd) * (1 - yd) + 
              (vx[xi, yi + 1, zi] * (1 - xd) + vx[xi + 1, yi + 1, zi] * xd) * yd) * (1 - zd) + \
             ((vx[xi, yi, zi + 1] * (1 - xd) + vx[xi + 1, yi, zi + 1] * xd) * (1 - yd) + 
              (vx[xi, yi + 1, zi + 1] * (1 - xd) + vx[xi + 1, yi + 1, zi + 1] * xd) * yd) * zd
    
    vy_val = ((vy[xi, yi, zi] * (1 - xd) + vy[xi + 1, yi, zi] * xd) * (1 - yd) + 
              (vy[xi, yi + 1, zi] * (1 - xd) + vy[xi + 1, yi + 1, zi] * xd) * yd) * (1 - zd) + \
             ((vy[xi, yi, zi + 1] * (1 - xd) + vy[xi + 1, yi, zi + 1] * xd) * (1 - yd) + 
              (vy[xi, yi + 1, zi + 1] * (1 - xd) + vy[xi + 1, yi + 1, zi + 1] * xd) * yd) * zd
    
    vz_val = ((vz[xi, yi, zi] * (1 - xd) + vz[xi + 1, yi, zi] * xd) * (1 - yd) + 
              (vz[xi, yi + 1, zi] * (1 - xd) + vz[xi + 1, yi + 1, zi] * xd) * yd) * (1 - zd) + \
             ((vz[xi, yi, zi + 1] * (1 - xd) + vz[xi + 1, yi, zi + 1] * xd) * (1 - yd) + 
              (vz[xi, yi + 1, zi + 1] * (1 - xd) + vz[xi + 1, yi + 1, zi + 1] * xd) * yd) * zd

    return np.array([vx_val, vy_val, vz_val])

# Function to compute streamlines given a set of starting points
def compute_streamlines(vx, vy, vz, x_range, y_range, z_range, start_points, max_distance=1e7, tol=1e-5):
    # Define the velocity field function for the integrator
    def velocity_field(t, pos):
        return velocity_field_numba(pos, vx, vy, vz, x_range, y_range, z_range)

    # Event to stop integration when a streamline hits the surface z = 0
    def boundary_event(t, pos):
        return pos[2] + 0
    
    boundary_event.terminal = True
    boundary_event.direction = 0

    # Function to integrate a single streamline
    def integrate_streamline(start):
        # Integrate forward in time
        result_forward = solve_ivp(velocity_field, [0, max_distance], start, method='RK45', rtol=tol, atol=tol, events=boundary_event)
        # Integrate backward in time
        result_backward = solve_ivp(velocity_field, [0, -max_distance], start, method='RK45', rtol=tol, atol=tol, events=boundary_event)
        # Combine forward and backward trajectories
        streamline = np.vstack((result_backward.y.T[::-1], result_forward.y.T))
        return streamline
    
    # Parallelize the integration of multiple streamlines
    with ThreadPoolExecutor() as executor:
        streamlines = list(executor.map(integrate_streamline, start_points))
    
    return streamlines

# Function to plot streamlines and starting points in a 3D space
def plot_streamlines_and_points(fig, streamlines, start_points, color):
    # Plot each streamline
    for streamline in streamlines:
        if len(streamline) > 0:
            fig.add_trace(go.Scatter3d(
                x=streamline[:, 0], y=streamline[:, 1], z=streamline[:, 2],
                mode='lines',
                line=dict(color=color, width=4)  # Set streamline color and width
            ))
    
    # Plot the starting points
    fig.add_trace(go.Scatter3d(
        x=start_points[:, 0], y=start_points[:, 1], z=start_points[:, 2],
        mode='markers',
        marker=dict(color=color, size=5, symbol='square'),
        name=f'Start Points {color}'
    ))

# Function to read starting points from a CSV file
def read_trace_points(file_path):
    df = pd.read_csv(file_path)
    return df[['x', 'y', 'z']].values

# Get the current working directory
current_working_dir = os.getcwd()

# Define the result folder path
result_folder = os.path.join(current_working_dir, 'results')
os.makedirs(result_folder, exist_ok=True)

# Define the input HDF5 file path
input_path_hdf5 = os.path.join(result_folder, 'head_and_velocity.h5')

# Read data from the HDF5 file
vx, vy, vz, UU, x_range, y_range, z_range = read_data_hdf5(input_path_hdf5)

# Define paths to the trace point CSV files
trace_point_files = [
    os.path.join(result_folder, 'best_y_trace_x-.csv'),
    os.path.join(result_folder, 'best_y_trace_x+.csv'),    
    os.path.join(result_folder, 'best_y_trace_y-.csv'),
    os.path.join(result_folder, 'best_y_trace_y+.csv'),
    os.path.join(result_folder, 'best_y_trace_z-.csv'),
    os.path.join(result_folder, 'best_y_trace_z+.csv')
]

# Define a list of colors for the streamlines
colors = ['darkgreen', 'darkviolet', 'lime', 'magenta', 'blue', 'red']

# Plot streamlines for each trace point file
for file_path, color in zip(trace_point_files, colors):
    # Create a new figure for each trace point file
    fig = go.Figure()

    # Read starting points
    start_points = read_trace_points(file_path)
    
    # Compute streamlines
    streamlines = compute_streamlines(vx, vy, vz, x_range, y_range, z_range, start_points)
    
    # Plot streamlines and starting points
    plot_streamlines_and_points(fig, streamlines, start_points, color)

    # Update the layout to adjust the axis ranges and aspect ratio
    fig.update_layout(
        scene=dict(
            xaxis=dict(range=[0, 10000], title='X Axis'),
            yaxis=dict(range=[0, 10000], title='Y Axis'),
            zaxis=dict(range=[-2000, 0], title='Z Axis'),
            aspectmode='cube'  # Maintain aspect ratio for all axes
        ),
        title=f'Streamlines for {os.path.basename(file_path)}',
        showlegend=False
    )

    # Define the output file path for the HTML visualization
    output_html_path = os.path.join(result_folder, f'{os.path.basename(file_path).replace(".csv", "")}_streamlines.html')

    # Display the figure
    fig.show()

    # Save the figure as an HTML file
    fig.write_html(output_html_path)

    # Printf-style output for current processing file
    print(f"Processed file: {file_path} with color {color}")

# Final notification after processing all files
print("Streamline computation and visualization completed for all trace point files.")



In [ ]:
# Step 4: Merge Dividing Streamlines and Show Groundwater Flow Systems 
import numpy as np
import h5py
import os
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio
from scipy.integrate import solve_ivp
from concurrent.futures import ThreadPoolExecutor
from numba import jit

# Function to read data from an HDF5 file
def read_data_hdf5(filename):
    with h5py.File(filename, 'r') as f:
        vx = f['vx'][:]
        vy = f['vy'][:]
        vz = f['vz'][:]
        UU = f['UU'][:]
        x_range = f['x_range'][:]
        y_range = f['y_range'][:]
        z_range = f['z_range'][:]
    return vx, vy, vz, UU, x_range, y_range, z_range

# JIT-compiled velocity field function
@jit(nopython=True)
def velocity_field_numba(pos, vx, vy, vz, x_range, y_range, z_range):
    x, y, z = pos
    if not (x_range[0] <= x <= x_range[-1]) or not (y_range[0] <= y <= y_range[-1]) or not (z_range[0] <= z <= z_range[-1]):
        return np.array([0.0, 0.0, 0.0])

    xi = np.searchsorted(x_range, x) - 1
    yi = np.searchsorted(y_range, y) - 1
    zi = np.searchsorted(z_range, z) - 1

    xd = (x - x_range[xi]) / (x_range[xi + 1] - x_range[xi])
    yd = (y - y_range[yi]) / (y_range[yi + 1] - y_range[yi])
    zd = (z - z_range[zi]) / (z_range[zi + 1] - z_range[zi])

    vx_val = ((vx[xi, yi, zi] * (1 - xd) + vx[xi + 1, yi, zi] * xd) * (1 - yd) + 
              (vx[xi, yi + 1, zi] * (1 - xd) + vx[xi + 1, yi + 1, zi] * xd) * yd) * (1 - zd) + \
             ((vx[xi, yi, zi + 1] * (1 - xd) + vx[xi + 1, yi, zi + 1] * xd) * (1 - yd) + 
              (vx[xi, yi + 1, zi + 1] * (1 - xd) + vx[xi + 1, yi + 1, zi + 1] * xd) * yd) * zd

    vy_val = ((vy[xi, yi, zi] * (1 - xd) + vy[xi + 1, yi, zi] * xd) * (1 - yd) + 
              (vy[xi, yi + 1, zi] * (1 - xd) + vy[xi + 1, yi + 1, zi] * xd) * yd) * (1 - zd) + \
             ((vy[xi, yi, zi + 1] * (1 - xd) + vy[xi + 1, yi, zi + 1] * xd) * (1 - yd) + 
              (vy[xi, yi + 1, zi + 1] * (1 - xd) + vy[xi + 1, yi + 1, zi + 1] * xd) * yd) * zd

    vz_val = ((vz[xi, yi, zi] * (1 - xd) + vz[xi + 1, yi, zi] * xd) * (1 - yd) + 
              (vz[xi, yi + 1, zi] * (1 - xd) + vz[xi + 1, yi + 1, zi] * xd) * yd) * (1 - zd) + \
             ((vz[xi, yi, zi + 1] * (1 - xd) + vz[xi + 1, yi, zi + 1] * xd) * (1 - yd) + 
              (vz[xi, yi + 1, zi + 1] * (1 - xd) + vz[xi + 1, yi + 1, zi + 1] * xd) * yd) * zd

    return np.array([vx_val, vy_val, vz_val])

# Function to compute streamlines
def compute_streamlines(vx, vy, vz, x_range, y_range, z_range, start_points, max_distance=1e7, tol=1e-5):
    def velocity_field(t, pos):
        return velocity_field_numba(pos, vx, vy, vz, x_range, y_range, z_range)

    def boundary_event(t, pos):
        return pos[2] + 0  # Termination condition, z = 0 (surface)
    
    boundary_event.terminal = True
    boundary_event.direction = 0

    def integrate_streamline(start):
        result_forward = solve_ivp(velocity_field, [0, max_distance], start, method='RK45', rtol=tol, atol=tol, events=boundary_event)
        result_backward = solve_ivp(velocity_field, [0, -max_distance], start, method='RK45', rtol=tol, atol=tol, events=boundary_event)
        
        streamline = np.vstack((result_backward.y.T[::-1], result_forward.y.T))
        return streamline
    
    with ThreadPoolExecutor() as executor:
        streamlines = list(executor.map(integrate_streamline, start_points))
    
    return streamlines

# Function to plot streamlines and trace points
def plot_streamlines_and_points(fig, streamlines, start_points, color):
    for streamline in streamlines:
        if len(streamline) > 0:
            fig.add_trace(go.Scatter3d(
                x=streamline[:, 0], y=streamline[:, 1], z=streamline[:, 2],
                mode='lines',
                line=dict(color=color, width=4)  # Set streamline color and width
            ))
    
    # Plot trace points
    fig.add_trace(go.Scatter3d(
        x=start_points[:, 0], y=start_points[:, 1], z=start_points[:, 2],
        mode='markers',
        marker=dict(color=color, size=5, symbol='square'),
        name=f'Start Points {color}'
    ))

# Function to read trace points from CSV
def read_trace_points(file_path):
    df = pd.read_csv(file_path, nrows=5)
    
    # Check the number of columns and read the complete file accordingly
    if df.shape[1] == 4:
        df = pd.read_csv(file_path, names=['x', 'y', 'z', 'UU'], header=0, skiprows=1)
    elif df.shape[1] == 3:
        df = pd.read_csv(file_path, names=['x', 'y', 'z'], header=0, skiprows=1)
    else:
        raise ValueError("CSV file format is incorrect, should contain 3 or 4 columns")
    
    return df[['x', 'y', 'z']].values

# Define paths and directories
current_working_dir = os.getcwd()
result_folder = os.path.join(current_working_dir, 'results')
os.makedirs(result_folder, exist_ok=True)
input_path_hdf5 = os.path.join(result_folder, 'head_and_velocity.h5')

# Read data from HDF5 file
vx, vy, vz, UU, x_range, y_range, z_range = read_data_hdf5(input_path_hdf5)

# Define trace point file paths
trace_point_files = [
    os.path.join(result_folder, 'best_y_trace_x-.csv'),
    os.path.join(result_folder, 'best_y_trace_x+.csv'),
    os.path.join(result_folder, 'best_y_trace_z-.csv'),
    os.path.join(result_folder, 'best_y_trace_z+.csv')
]

# Colors for different trace point sets
colors = ['lime', 'magenta', 'blue', 'red']

# Create a 3D figure
fig = go.Figure()

# Read "best_y.csv" file and plot its points
best_y_path = os.path.join(result_folder, 'best_y.csv')
best_y_points = read_trace_points(best_y_path)

# Read each trace point file, compute streamlines, and plot them
for file_path, color in zip(trace_point_files, colors):
    start_points = read_trace_points(file_path)
    streamlines = compute_streamlines(vx, vy, vz, x_range, y_range, z_range, start_points)
    plot_streamlines_and_points(fig, streamlines, start_points, color)

# Add the best_y points to the figure as black hollow circles
fig.add_trace(go.Scatter3d(
    x=best_y_points[:, 0], y=best_y_points[:, 1], z=best_y_points[:, 2],
    mode='markers',
    marker=dict(color='black', size=6, symbol='circle'),
    name='Best Y Points'
))

# Update layout to adjust display range
fig.update_layout(
    scene=dict(
        xaxis=dict(range=[0, 10000], title='X Axis'),
        yaxis=dict(range=[0, 10000], title='Y Axis'),
        zaxis=dict(range=[-2000, 0], title='Z Axis'),
        aspectmode='manual',  # Allow custom aspect ratio
        aspectratio=dict(x=1, y=1, z=0.5)  # Custom aspect ratio for display range
    ),
    margin=dict(l=0, r=0, b=0, t=0)  # Remove margins
)

# Save the figure as an HTML file
output_html_file = os.path.join(result_folder, '3D_GFSs_Case3S.html')
pio.write_html(fig, file=output_html_file, auto_open=False)

# Display the figure
fig.show()

